In [1]:
import polars as pl
import pyarrow.compute as pc

### Country codes and names

In [3]:
country_codes = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv"
raw_country_codes = pl.read_csv(country_codes)

country_codes = raw_country_codes.select(
    pl.col("name").alias("country_name"), 'alpha-2', 'alpha-3',
    pl.col('name').str.to_lowercase().str.strip_chars().str.replace(" ", "_", n=-1).alias('clean_name'),
).with_columns(
    # Attempting to remove accesnts. doesnt work too well
    # https://github.com/pola-rs/polars/issues/11455
    pl.col("clean_name").map_batches(lambda x: pl.from_arrow(pc.utf8_normalize(x.to_arrow(), "NFKD")))
)

country_codes.write_csv("../../data/processed/country_codes.csv")
country_codes

country_name,alpha-2,alpha-3,clean_name
str,str,str,str
"""Afghanistan""","""AF""","""AFG""","""afghanistan"""
"""Åland Islands""","""AX""","""ALA""","""åland_islands"""
"""Albania""","""AL""","""ALB""","""albania"""
"""Algeria""","""DZ""","""DZA""","""algeria"""
"""American Samoa""","""AS""","""ASM""","""american_samoa"""
…,…,…,…
"""Wallis and Futuna""","""WF""","""WLF""","""wallis_and_futuna"""
"""Western Sahara""","""EH""","""ESH""","""western_sahara"""
"""Yemen""","""YE""","""YEM""","""yemen"""


### Australian post codes

In [2]:
australian_postcodes = "https://raw.githubusercontent.com/Elkfox/Australian-Postcode-Data/refs/heads/master/au_postcodes.csv"
raw_post_codes = pl.read_csv(australian_postcodes)
post_codes = raw_post_codes.select(
    pl.col("postcode"),
    pl.col('place_name').str.to_lowercase().str.strip_chars().str.replace(" ", "_", n=-1).alias('suburb'),
)
post_codes.write_csv("../../data/processed/AUS/postcodes.csv")
post_codes

postcode,suburb
i64,str
200,"""australian_national_university"""
221,"""barton"""
2540,"""wreck_bay"""
2540,"""hmas_creswell"""
2540,"""jervis_bay"""
…,…
6989,"""maddington"""
6990,"""gosnells"""
6991,"""kelmscott"""
